In [6]:
import requests
from datetime import datetime, timedelta
import os
from typing import Dict, List, Optional, Union
import polyline
from dotenv import load_dotenv

In [9]:
GOOGLE_MAPS_API_KEY = ''

In [17]:
class MapsDirectionsTool:
    """A tool for retrieving directions and travel information using Google Maps API."""
    
    def __init__(self, api_key: Optional[str] = None):
        """
        Initialize the MapsDirectionsTool.
        
        Args:
            api_key: Google Maps API key. If not provided, will try to get it from GOOGLE_MAPS_API_KEY env variable.
        """
        self.api_key = api_key or GOOGLE_MAPS_API_KEY
        
        if not self.api_key:
            raise ValueError(
                "Google Maps API key is required. Set the GOOGLE_MAPS_API_KEY environment variable "
                "or pass it directly to the MapsDirectionsTool constructor."
            )
        
        self.directions_url = "https://maps.googleapis.com/maps/api/directions/json"
    
    def get_directions(self, 
                      origin: str, 
                      destination: str, 
                      mode: str = "driving", 
                      departure_time: Optional[str] = None,
                      arrival_time: Optional[str] = None) -> Dict:
        """
        Get directions from origin to destination.
        
        Args:
            origin: Starting address or location
            destination: Ending address or location
            mode: Mode of transportation ('driving', 'walking', 'bicycling', 'transit')
            departure_time: Optional departure time in ISO format (YYYY-MM-DDTHH:MM:SS)
            arrival_time: Optional arrival time in ISO format (YYYY-MM-DDTHH:MM:SS)
            
        Returns:
            Dict containing directions information
        """
        params = {
            "origin": origin,
            "destination": destination,
            "mode": mode,
            "key": self.api_key
        }
        
        # Convert departure_time to UNIX timestamp if provided
        if departure_time:
            try:
                dt = datetime.fromisoformat(departure_time)
                params["departure_time"] = int(dt.timestamp())
            except ValueError:
                raise ValueError(f"Invalid departure_time format. Expected ISO format (YYYY-MM-DDTHH:MM:SS), got {departure_time}")
        
        # Convert arrival_time to UNIX timestamp if provided
        if arrival_time:
            try:
                at = datetime.fromisoformat(arrival_time)
                params["arrival_time"] = int(at.timestamp())
            except ValueError:
                raise ValueError(f"Invalid arrival_time format. Expected ISO format (YYYY-MM-DDTHH:MM:SS), got {arrival_time}")
        
        # For transit mode, we need either departure_time or arrival_time
        if mode == "transit" and not (departure_time or arrival_time):
            # Use current time as default departure time
            params["departure_time"] = int(datetime.now().timestamp())
        
        # Add alternatives to get multiple route options
        params["alternatives"] = "true"
        
        response = requests.get(self.directions_url, params=params)
        
        if response.status_code != 200:
            error_msg = response.json().get("error_message", "Unknown error")
            raise Exception(f"Error fetching directions: {error_msg} (Code: {response.status_code})")
        
        result = response.json()
        
        if result["status"] != "OK":
            error_msg = result.get("error_message", result["status"])
            raise Exception(f"Error fetching directions: {error_msg}")
        
        return result
    
    def extract_route_info(self, directions_data: Dict, route_index: int = 0) -> Dict:
        """
        Extract useful information from a specific route in the directions data.
        
        Args:
            directions_data: Data returned from get_directions
            route_index: Index of the route to extract info from (default: 0 for primary route)
            
        Returns:
            Dict with simplified route information
        """
        if not directions_data.get("routes"):
            raise ValueError("No routes found in directions data")
        
        if route_index >= len(directions_data["routes"]):
            raise ValueError(f"Route index {route_index} out of range. Only {len(directions_data['routes'])} routes available.")
        
        route = directions_data["routes"][route_index]
        legs = route["legs"]
        
        # Extract overall distance and duration
        total_distance = sum(leg["distance"]["value"] for leg in legs)  # in meters
        total_duration = sum(leg["duration"]["value"] for leg in legs)  # in seconds
        
        # Check if there's traffic duration information
        has_traffic_info = any("duration_in_traffic" in leg for leg in legs)
        if has_traffic_info:
            total_duration_in_traffic = sum(leg.get("duration_in_traffic", {}).get("value", leg["duration"]["value"]) for leg in legs)
        else:
            total_duration_in_traffic = None
        
        # Extract start and end addresses
        start_address = legs[0]["start_address"]
        end_address = legs[-1]["end_address"]
        
        # Extract steps for navigation
        steps = []
        for leg_index, leg in enumerate(legs):
            for step in leg["steps"]:
                step_info = {
                    "instruction": step["html_instructions"],
                    "distance": step["distance"]["text"],
                    "duration": step["duration"]["text"],
                }
                
                # Add transit-specific details if available
                if step.get("transit_details"):
                    transit = step["transit_details"]
                    step_info["transit"] = {
                        "line": transit.get("line", {}).get("name", "Unknown line"),
                        "departure_stop": transit.get("departure_stop", {}).get("name", "Unknown"),
                        "arrival_stop": transit.get("arrival_stop", {}).get("name", "Unknown"),
                        "num_stops": transit.get("num_stops", "Unknown"),
                        "departure_time": transit.get("departure_time", {}).get("text", "Unknown"),
                        "arrival_time": transit.get("arrival_time", {}).get("text", "Unknown"),
                    }
                
                steps.append(step_info)
        
        # Get polyline for the route
        route_polyline = route["overview_polyline"]["points"]
        
        # Return simplified route info
        return {
            "start_address": start_address,
            "end_address": end_address,
            "distance": {
                "text": self._format_distance(total_distance),
                "value": total_distance
            },
            "duration": {
                "text": self._format_duration(total_duration),
                "value": total_duration
            },
            "duration_in_traffic": {
                "text": self._format_duration(total_duration_in_traffic) if total_duration_in_traffic else None,
                "value": total_duration_in_traffic
            } if has_traffic_info else None,
            "steps": steps,
            "polyline": route_polyline,
        }
    
    def format_directions_response(self, route_info: Dict, include_steps: bool = True) -> str:
        """
        Format the route information into a human-readable string.
        
        Args:
            route_info: Route information from extract_route_info
            include_steps: Whether to include step-by-step directions
            
        Returns:
            Formatted directions text
        """
        # Start with summary information
        result = f"Directions from {route_info['start_address']} to {route_info['end_address']}:\n\n"
        result += f"Total Distance: {route_info['distance']['text']}\n"
        result += f"Estimated Travel Time: {route_info['duration']['text']}\n"
        
        # Add traffic information if available
        if route_info.get("duration_in_traffic"):
            result += f"Travel Time with Traffic: {route_info['duration_in_traffic']['text']}\n"
        
        # Add URL to Google Maps
        start_encoded = requests.utils.quote(route_info['start_address'])
        end_encoded = requests.utils.quote(route_info['end_address'])
        maps_url = f"https://www.google.com/maps/dir/?api=1&origin={start_encoded}&destination={end_encoded}"
        result += f"\nView on Google Maps: {maps_url}\n"
        
        # Add step-by-step directions if requested
        if include_steps and route_info.get("steps"):
            result += "\nStep-by-Step Directions:\n"
            
            for i, step in enumerate(route_info["steps"], 1):
                # Clean up HTML tags in instructions (simple approach)
                instruction = step["instruction"].replace("<b>", "").replace("</b>", "")
                instruction = instruction.replace("<div>", "\n  ").replace("</div>", "")
                
                result += f"{i}. {instruction} ({step['distance']} - {step['duration']})\n"
                
                # Add transit details if available
                if step.get("transit"):
                    transit = step["transit"]
                    result += f"   Take {transit['line']} from {transit['departure_stop']} to {transit['arrival_stop']}\n"
                    result += f"   Departure: {transit['departure_time']}, Arrival: {transit['arrival_time']}\n"
                    result += f"   {transit['num_stops']} stops\n"
                
        return result
    
    def _format_distance(self, distance_in_meters: float) -> str:
        """Format distance in a human-readable way."""
        if distance_in_meters < 1000:
            return f"{distance_in_meters:.0f} m"
        else:
            return f"{distance_in_meters/1000:.1f} km"
    
    def _format_duration(self, duration_in_seconds: float) -> str:
        """Format duration in a human-readable way."""
        if not duration_in_seconds:
            return "Unknown"
            
        hours, remainder = divmod(int(duration_in_seconds), 3600)
        minutes, seconds = divmod(remainder, 60)
        
        parts = []
        if hours > 0:
            parts.append(f"{hours} hr")
        if minutes > 0 or not parts:  # Always show minutes if there are no hours
            parts.append(f"{minutes} min")
            
        return " ".join(parts)


def get_directions(origin: str, destination: str, mode: str = "driving", 
                  departure_time: Optional[str] = None, arrival_time: Optional[str] = None,
                  include_steps: bool = True) -> str:
    """
    Get directions between two locations.
    
    Args:
        origin: Starting address or location
        destination: Ending address or location
        mode: Mode of transportation ('driving', 'walking', 'bicycling', 'transit')
        departure_time: Optional departure time in ISO format (YYYY-MM-DDTHH:MM:SS)
        arrival_time: Optional arrival time in ISO format (YYYY-MM-DDTHH:MM:SS)
        include_steps: Whether to include step-by-step directions
        
    Returns:
        String with formatted directions information
    """
    try:
        # Validate mode
        valid_modes = ["driving", "walking", "bicycling", "transit"]
        if mode not in valid_modes:
            return f"Invalid mode: {mode}. Please use one of {', '.join(valid_modes)}."
        
        # If mode is specified as "public", convert to "transit" for Google Maps API
        if mode.lower() == "public":
            mode = "transit"
            
        maps_tool = MapsDirectionsTool()
        
        # Get directions data
        directions_data = maps_tool.get_directions(
            origin=origin,
            destination=destination,
            mode=mode,
            departure_time=departure_time,
            arrival_time=arrival_time
        )
        
        # Extract route information
        route_info = maps_tool.extract_route_info(directions_data)
        
        # Format the response
        return maps_tool.format_directions_response(route_info, include_steps)
        
    except Exception as e:
        return f"Error retrieving directions: {str(e)}"


if __name__ == "__main__":
    # Test with basic directions
    print("DRIVING DIRECTIONS:")
    print(get_directions(
        origin="75 Saint Alphonsus Street Boston MA",
        destination="700 boylston street, boston, ma 02116",
        mode="transit"
    ))

DRIVING DIRECTIONS:
Directions from 75 St Alphonsus St, Boston, MA 02120, USA to 700 Boylston St, Boston, MA 02116, USA:

Total Distance: 2.8 km
Estimated Travel Time: 16 min

View on Google Maps: https://www.google.com/maps/dir/?api=1&origin=75%20St%20Alphonsus%20St%2C%20Boston%2C%20MA%2002120%2C%20USA&destination=700%20Boylston%20St%2C%20Boston%2C%20MA%2002116%2C%20USA

Step-by-Step Directions:
1. Walk to Longwood Medical Area (0.2 mi - 4 mins)
2. Light rail towards Medford/Tufts (1.5 mi - 11 mins)
   Take Green Line E from Longwood Medical Area to Copley
   Departure: 2:02 PM, Arrival: 2:12 PM
   5 stops
3. Walk to 700 Boylston St, Boston, MA 02116, USA (358 ft - 3 mins)

